(SMALL BUG) NOTE: if a chord of type '56' is selected (instead of what it should be: '5/6') then this won't work. easy to fix but not necessary since web (JS) version has values hard-coded.

In [ ]:
# import useful libraries:
import pandas as pd
import random

In [ ]:
CHORD_PROGRESSION_LENGTH = 4 # how many chords will the generated chord progression have?

In [ ]:
# import the CSV file:
CSV_URL = 'https://raw.githubusercontent.com/jhamer90811/chord_progression_assistant/master/four_chord_songs.csv'
data = pd.read_csv(CSV_URL)['cp'].tolist()

# process the data (separate strings by comma):
data = [d.split(',') for d in data]

In [ ]:
# analyze frequencies of starting chords:
first_chords = {}
for chord_progression in data:
  if chord_progression[0] not in first_chords:
    first_chords[chord_progression[0]] = 1
  else:
    first_chords[chord_progression[0]] += 1

# normalize data to get frequency percentage:
total_votes = sum(first_chords[i] for i in first_chords)
for chord in first_chords:
  first_chords[chord] /= total_votes

In [ ]:
# something resembling n-gram analysis:
next_chord = {}
for chord_progression in data:
  for c in range(len(chord_progression) - 1):
    if chord_progression[c] not in next_chord:
      next_chord[chord_progression[c]] = {}
    if chord_progression[c+1] not in next_chord[chord_progression[c]]:
      next_chord[chord_progression[c]][chord_progression[c+1]] = 1
    else:
      next_chord[chord_progression[c]][chord_progression[c+1]] += 1

In [ ]:
# normalize the data to get frequency percentage:
for starting_chord in next_chord:
  total_votes = sum(next_chord[starting_chord][k] for k in next_chord[starting_chord])
  for chord in next_chord[starting_chord]:
    next_chord[starting_chord][chord] /= total_votes

In [ ]:
# utility function to pick an element out of a map of weights (weighted random):
def pick(elements):
  # generate a random number from 0 to 1:
  r = random.random()

  # pick the number accordingly:
  for e in elements:
    if r < elements[e]:
      return e
    r -= elements[e]

In [ ]:
# generate a chord progression:
cp = [pick(first_chords)] # pick the first chord
for i in range(CHORD_PROGRESSION_LENGTH-1):
  cp.append(pick(next_chord[cp[-1]]))

In [ ]:
# useful lists for storing musical information:
keys = ['A', 'Bb', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#']
chord_type = ['-', 'maj', 'min', 'min', 'maj', 'maj', 'min', 'dim']
translate = {'1' : 0, '2' : 2, '3' : 4, '4': 5, '5': 7, '6': 9, '7': 11} # the major scale in semitones

# pick a key:
key = random.choice(keys)
key_index = keys.index(key)

# generate the chord progression in the selected key:
print('key: %s major' % key)
print('chord progression: ', end='')
for chord in cp:
  if '/' in chord:
    # special case, if chord is a two-part chord:
    chord_one, chord_two = chord.split('/')
    print('%s%s/' % (keys[(key_index + translate[chord_one]) % 12], chord_type[int(chord_one)]), end='')
    print('%s%s ' % (keys[(key_index + translate[chord_two]) % 12], chord_type[int(chord_two)]), end='')
  else:
    print('%s%s ' % (keys[(key_index + translate[chord]) % 12], chord_type[int(chord)]), end='')
print()
print('it\'s a %sprogression!' % ''.join(chord + ' ' for chord in cp))

key: F major
chord progression: Fmaj Bbmaj/Cmaj Cmaj Dmin/Edim 
it's a 1 4/5 5 6/7 progression!
